# quick_pp

In [ ]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from quick_pp.objects import Project

# Load well from saved file
project_name = "MOCK_carbonate"
project_path = rf"data\04_project\{project_name}.qppp"
project = Project().load(project_path)
project.get_well_names()

# Batch Interpretation

In [ ]:
import pandas as pd
import numpy as np
import pickle

from quick_pp.lithology.carbonate import Carbonate
from quick_pp.porosity import neu_den_xplot_poro, density_porosity, rho_matrix
from quick_pp.qaqc import badhole_flagging, mask_outside_threshold, neu_den_xplot_hc_correction
from quick_pp.saturation import estimate_temperature_gradient, estimate_rw_temperature_salinity, archie_saturation
from quick_pp.ressum import calc_reservoir_summary
from quick_pp.rock_type import estimate_vsh_gr, calc_r35_perm, rock_typing, calc_fzi_perm
from quick_pp.core_calibration import sw_shf_leverett_j, sw_skelt_harrison

# Load ROCK_FLAG
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_rt_model.qppm', 'rb') as file:
    fzi_rt_model = pickle.load(file)

# Load R35 model
with open(r'data\04_project\MOCK_carbonate\outputs\r35_model.qppm', 'rb') as file:
    r35_model = pickle.load(file)

# Load FZI model
with open(r'data\04_project\MOCK_carbonate\outputs\fzi_model.qppm', 'rb') as file:
    fzi_model = pickle.load(file)

In [ ]:
WATER_CONTACT = {
    'HW-3': 8220,
    'HW-4': 8220,
    'HW-5': 8243,
    'HW-6': 8082,
    'HW-7': 8533,
    'HW-8': 8658,
    'HW-9': 8658,
    'HW-10': 8554,
    'HW-24': 8552,
    'HW-25': 8421,
    'HW-26': 8573,
    'HW-27': 8656,
    'HW-28': 8564,
    'HW-29': 8377,
    'HW-30': 8543,
    'HW-31': 8594,
    'HW-32': 8543,
}

In [ ]:
carbonate_type =  'limestone'  # 'dolostone'  #
model = 'single'  # 'double'  #
method = 'neu_den'  # 'pef_den'  #

df = project.get_all_data()  # [['WELL_NAME', 'DEPTH', 'ZONES', 'GR', 'RT', 'NPHI', 'RHOB', 'CALI', 'BS']]
# df = all_data.copy()
for well_name, well_data in df.groupby('WELL_NAME'):
    # Load well object
    well = project.get_well(well_name)
    well_data['CPORE'] = well_data['CORE_POR'] / 100
    well_data['CPERM'] = well_data['CORE_PERM']

    # Clean up data
    well_data = badhole_flagging(well_data)
    # well_data = mask_outside_threshold(well_data, fill=True)

    # Initialize lithology model
    args = {
        'litho_model': 'carb',
        'dry_calc_point': (.0, 2.71),
        'dry_clay_point': (.3, 2.7),
        'sw_water_salinity': 2e5,
        'sw_m': 2.2,
        'sw_n': 2,
        'hc_corr_angle': 50,
        'hc_buffer': 0.1,
        'ressum_cutoffs': dict(
            VSHALE=.5,
            PHIT=0,
            SWT=1
        ),
    }
    vsh_gr = estimate_vsh_gr(well_data['GR'], min_gr=0, max_gr=200)
    carb_model = Carbonate(**args)
    vclw, vcalc, vdolo = carb_model.estimate_lithology(
        well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], vsh_gr=vsh_gr, # pef=well_data['PEF'],
        model=model, method=method, normalize= True, carbonate_type=carbonate_type
    )
    args.update(carb_model.__dict__)
    well.update_config(args)  # Save lithology model to well

    # Set skip HC correction or not
    skip_hc_correction = False
    if skip_hc_correction:
        nphihc, rhobhc = well_data['NPHI'], well_data['RHOB']
    else:
        # Implement hydrocarbon correction
        nphihc, rhobhc, hc_flag = neu_den_xplot_hc_correction(
            well_data['NPHI'], well_data['RHOB'], gr=well_data['GR'], vsh_gr=vsh_gr,
            dry_min1_point=args['dry_calc_point'],
            dry_clay_point=args['dry_clay_point'],
            corr_angle=args['hc_corr_angle'], buffer=args['hc_buffer']
        )

        # Estimate lithology
        carb_model = Carbonate(**args)
        vclw, vcalc, vdolo = carb_model.estimate_lithology(
            nphihc, rhobhc,  gr=well_data['GR'], vsh_gr=vsh_gr, # pef=well_data['PEF'],
            model=model, method=method, normalize= True, carbonate_type=carbonate_type
        )

    # Estimate porosity
    phit = neu_den_xplot_poro(
        nphihc, rhobhc, model='carb',
        dry_min1_point=args['dry_calc_point'],
        dry_clay_point=args['dry_clay_point'],
    )

    # PHID needs unnormalized lithology
    vclw_un, vcalc_un, vdolo_un = Carbonate(**args).estimate_lithology(
        nphihc, rhobhc, gr=well_data['GR'], vsh_gr=vsh_gr, # pef=well_data['PEF'],
        model=model, method=method, normalize= False, carbonate_type=carbonate_type
    )
    rho_ma = rho_matrix(vclay=vclw_un, vcalc=vcalc_un, vdolo=vdolo_un)
    phid = density_porosity(rhobhc, rho_ma)
    
    # Fill missing values in phit with phid
    phit = np.where(np.isnan(phit), phid, phit)
    
    well_data['LOG_RT'] = np.log10(well_data['RT'])
    well_data['NDI'] = (2.95 - well_data['RHOB']) / 1.95
    # well_data['NDI_V2'] = np.log10((2.95 - well_data['RHOB']) / 1.95)
    input_features = ['GR', 'NPHI', 'RHOB', 'LOG_RT', 'NDI']
    
    use_r35 = True
    use_j = False
    if use_r35:
        # Predict permeability
        r35_ml = 10**(r35_model.predict(well_data[input_features]))
        perm_ml = calc_r35_perm(r35_ml, phit)

        # Determine rock type based on perm_r35_ml
        r35_cut_offs = [
            0.15, .2, .3, .4, .5, .6, .7, .8, .9, 1, 1.25, 1.5, 2, 2.5, 3, 4, 5, 7, 10,
        ]
        rock_flag_ml = rock_typing(r35_ml, higher_is_better=True, cut_offs=r35_cut_offs)
        if use_j:
            shf_params = {
                1: (0.006, 1.212),
                2: (0.004, 1.733),
                3: (0.008, 1.465),
                4: (0.012, 1.513),
                5: (0.005, 1.6),
                6: (0.015, 0.688),
                7: (0.016, 0.932),
                8: (0.024, 0.585),
                9: (0.002, 2.136),
                10: (0.015, 1.052),
                11: (0.013, 0.819),
                12: (0.006, 2.057),
                13: (0.012, 0.861),
                14: (0.012, 0.797),
                15: (0.018, 1.457),
                16: (0.007, 1.21),
                17: (0.026, 0.773),
                18: (0.018, 0.927),
                19: (0.021, 0.649),
                20: (0.026, 1.033),
                21: (0.023, 0.922),
                22: (0.008, 1.279),
                23: (0.025, 0.581)
            }
        else:
            shf_params = {
                1: (0.984, 4.21, 1.375, 0.013),
                2: (0.995, 4.719, 0.67, -8.917),
                3: (0.989, 6.811, 0.863, -5.241),
                4: (1.0, 6.22, 0.684, -0.684),
                5: (0.982, 5.875, 0.936, -9.54),
                6: (1.001, 16.434, 1.16, -32.532),
                7: (1.009, 15.35, 1.011, -2.99),
                8: (0.986, 2.057, 0.986, -16.31),
                9: (0.973, 5.123, 0.978, -10.99),
                10: (0.973, 15.095, 1.67, -0.43),
                11: (0.978, 6.044, 1.524, -12.53),
                12: (0.962, 10.955, 0.766, -5.951),
                13: (0.99, 11.986, 2.283, 4.623),
                14: (0.988, 11.103, 1.498, -9.238),
                15: (0.999, 23.941, 0.759, -4.405),
                16: (0.972, 6.367, 1.003, -25.049),
                17: (1.005, 15.207, 1.163, -3.528),
                18: (0.99, 10.779, 1.086, -11.725),
                19: (0.994, 52.826, 2.571, 7.828),
                20: (0.983, 73.296, 1.227, -17.752),
                21: (1.009, 27.135, 0.92, -7.199),
                22: (0.963, 41.903, 2.026, -15.249),
                23: (0.981, 50.296, 2.092, -17.645)
            }
    else:
        # Predict permeability
        fzi_ml = 10**(fzi_model.predict(well_data[input_features]))
        perm_ml = calc_fzi_perm(fzi_ml, phit)

        # Determine rock type based on perm_fzi_ml
        fzi_cut_offs = [
            .15, .2, .25, .3, .35, .4, .45, .5, .6, .7, .8, .9, 1.1, 1.3, 1.5, 2, 2.5, 3, 4, 5, 7, 10
        ]
        rock_flag_ml = rock_typing(fzi_ml, higher_is_better=True, cut_offs=fzi_cut_offs)
        if use_j:
            shf_params = {   
                1: (0.009, 1.033),
                2: (0.026, 0.978),
                3: (0.006, 1.578),
                4: (0.022, 0.566),
                5: (0.032, 0.387),
                6: (0.022, 0.513),
                7: (0.015, 0.659),
                8: (0.018, 0.728),
                9: (0.02, 0.85),
                10: (0.033, 0.572),
                11: (0.03, 0.52),
                12: (0.024, 0.761),
                13: (0.025, 0.658),
                14: (0.028, 0.513),
                15: (0.028, 0.579),
                16: (0.008, 1.615),
                17: (0.012, 1.136),
                18: (0.009, 1.494),
                19: (0.021, 0.944),
                20: (0.036, 0.493),
                21: (0.013, 0.853),
                22: (0.013, 1.121),
                23: (0.012, 1.341)
            }
        else:
            shf_params = {   
                1: (0.984, 4.21, 1.375, 0.013),
                2: (0.989, 6.811, 0.863, -5.241),
                3: (0.984, 11.11, 2.972, -2.043),
                4: (0.993, 3.068, 1.43, -3.204),
                5: (1.002, 4.197, 0.999, -14.705),
                6: (0.978, 6.044, 1.524, -12.53),
                7: (0.981, 8.302, 1.23, -5.811),
                8: (0.962, 10.955, 0.766, -5.951),
                9: (0.99, 10.779, 1.086, -11.725),
                10: (0.967, 11.867, 0.692, -11.725),
                11: (1.028, 117.22, 0.878, -35.702),
                12: (1.016, 16.947, 0.753, -2.373),
                13: (0.965, 22.599, 1.366, -7.533),
                14: (0.997, 11.854, 1.302, -17.048),
                15: (0.982, 12.346, 1.407, -27.404),
                16: (0.995, 29.102, 1.868, -17.645),
                17: (0.963, 41.903, 2.026, -15.249),
                18: (0.981, 50.296, 2.092, -17.645),
                19: (0.983, 73.296, 1.227, -17.752),
                20: (0.985, 87.589, 0.658, -60.135),
                21: (0.946, 50.66, 0.966, -113.086),
                22: (0.952, 193.437, 1.317, -96.524),
                23: (0.933, 372.742, 1.397, -57.259)
            }

    # Estimate water saturation
    temp_grad = estimate_temperature_gradient(well_data['DEPTH'], 'metric')  # Arbitrarily use MD instead of TVD
    water_salinity = args['sw_water_salinity']
    rw = estimate_rw_temperature_salinity(temp_grad, water_salinity)
    m = args['sw_m']
    swt = archie_saturation(well_data['RT'], rw, phit, m=m)
    swt = swt.clip(0, 1)
    
    # Estimate SHF
    ift = 32
    theta = 30
    ghc = .837
    gw = 1.135

    fwl = WATER_CONTACT[well_name]

    if use_j:
        a = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[0])
        b = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[1])
        shf = sw_shf_leverett_j(
            perm_ml, phit, well_data['DEPTH'], gw=gw, ghc=ghc,
            fwl=fwl, ift=ift, theta=theta, a=a, b=b
        )
    else:
        a = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[0])
        b = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[1])
        c = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[2])
        d = pd.Series(rock_flag_ml).map(shf_params).apply(lambda x: x[3])
        shf = sw_skelt_harrison(well_data.DEPTH, fwl, a, b, c, d)

    # Estimate reservoir summary
    well_data['ZONES'] = 'ALL'
    ressum_df = calc_reservoir_summary(well_data.DEPTH, vclw, phit, swt, perm_ml,
                                       zones=well_data['ZONES'], cutoffs=args['ressum_cutoffs'])

    # Update data in the project
    well_data['NPHI_HC'] = nphihc
    well_data['RHOB_HC'] = rhobhc
    # well_data['HC_FLAG'] = hc_flag
    well_data['VCALC'] = vcalc
    well_data['VDOLO'] = vdolo
    well_data['VCLW'] = vclw
    well_data['PHID'] = phid
    well_data['PHIT'] = phit
    well_data['SWT'] = swt
    well_data['SHF'] = shf
    well_data['BVW'] = phit * swt
    well_data['PERM'] = perm_ml
    well_data['VHC'] = phit * (1 - swt)
    well_data['ROCK_FLAG'] = rock_flag_ml
    well_data['ROCK_FLAG'] = well_data['ROCK_FLAG'].astype(int)

    # Save the results to the well
    well.update_data(well_data)
    well.update_ressum(ressum_df)
    project.save_well(well)
    # break


In [ ]:
from quick_pp.plotter import plotly_log
import os

df = project.get_all_data()
folder = rf"outputs\{project_name}\interpreted_logs"
os.makedirs(folder, exist_ok=True)
for well, well_data in df.groupby("WELL_NAME"):
    # Plot the results
    fig = plotly_log(well_data, depth_uom='ft')
    # fig.show(config=dict(scrollZoom=True))
    fig.write_html(rf"{folder}\{well}_log.html", config=dict(scrollZoom=True))

# QC the results

In [ ]:
df = project.get_all_data()

In [ ]:
df.columns

In [ ]:
from quick_pp.qaqc import quick_compare
from quick_pp.utils import zone_flagging

df['VSHALE'] = df['VCLW']
df = zone_flagging(df)
compare_df, fig = quick_compare(df, return_fig=True)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error

vars_dict = {
    'PHIT': 'CPORE',
    'PERM': 'CPERM',
    'SWT': 'SHF'
}

for var, compare_var in vars_dict.items():
    data = df[[var, compare_var]].dropna()
    r2 = r2_score(data[compare_var], data[var])
    mape = mean_absolute_percentage_error(data[compare_var], data[var])
    mae = mean_absolute_error(data[compare_var], data[var])
    print(f"{var} R2: {r2:.2f}, MAPE: {mape:.2f}, MAE: {mae:.2f}")

results = []
for well_name, well_data in df.groupby('WELL_NAME'):
    well_results = {'WELL_NAME': well_name}
    for var, compare_var in vars_dict.items():
        data = well_data[[var, compare_var]].dropna()
        r2 = r2_score(data[compare_var], data[var])
        mape = mean_absolute_percentage_error(data[compare_var], data[var])
        mae = mean_absolute_error(data[compare_var], data[var])
        well_results[f"{var}_R2"] = r2
        well_results[f"{var}_MAPE"] = mape
        well_results[f"{var}_MAE"] = mae
    results.append(well_results)

results_df = pd.DataFrame(results).sort_values('PHIT_MAE')
results_df